# Exercise 09

In this exercise, we will further explore graph representation learning approaches and, in particular, Node2Vec. If you have questions regarding the exercise please use the WueCampus forum or contact Moritz Lampert via email (moritz.lampert@uni-wuerzburg.de). The exercise is due on July 9th 14:00 (UTC+2)

## Setup

The exercise uses the Python Library `PathPyG` that is based on the deep learning library [PyTorch](https://pytorch.org/docs/stable/index.html) and the graph learning library [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/). `PathPyG` enables interactive graph visualisations and implements a variety of algorithms for graphs and temporal graphs. For further information and installation instructions, see [pathpy.net](https://www.pathpy.net/0.1.0-dev/getting_started/) or the Jupyter Notebooks of Week 1 of the [GitLab repository](https://gitlab.informatik.uni-wuerzburg.de/ml4nets_notebooks/2024_sose_ml4nets_notebooks) that accompanies our Lecture Machine Learning for Complex Networks (ML4Nets) that was already used in the practice sessions of the lecture.


In [ ]:
import matplotlib.pyplot as plt
from numpy import array

from sklearn.decomposition import TruncatedSVD

from torch import tensor
from torch.nn import Module

from torch_geometric.data import Data

from pathpyG import Graph, plot, IndexMap
from pathpyG.processes import RandomWalk
from pathpyG.io import read_netzschleuder_network

## Introduction to `PathPyG`

In the following, you can find a brief introduction into PyTorch Geometric (PyG) and PathPyG. For more information refer to the tutorials in the corresponding documentations or the aforementioned notebooks of our ML4Nets Course.

### 1. Graphs in `PyG`

Mathematically, the edges of a graph are commonly represented as adjacency matrix $A \in \{0,1\}^{n\times n}$ where $n$ is the number of nodes. An element $A_{ij}$ is 1 iff the nodes $v_i$ and $v_j$ are connected via an edge. For large-scale graphs, this representation is not very space-efficient since real-world graphs are typically very sparse (most entries are 0). Thus, the default edge representation that is used in `PyG` is not an adjacency matrix but a so-called `edge_index` instead. An `edge_index` is a `Tensor` with shape `(2, m)` where $m$ is the number of edges in the graph and contains for each edge $e_i=(v_j, v_k)$ the indices `edge_index[0, i] = j` and `edge_index[1, i] = k`.

Let's create an example:

In [ ]:
edge_index = tensor([
    [0, 0, 1, 1, 3, 3, 3, 4, 5],
    [1, 2, 3, 2, 5, 4, 6, 5, 6]
])
print(f"Source Node of edge 2: {edge_index[0, 1]}")
print(f"Destination Node of edge 2: {edge_index[1, 1]}")

This is the most basic graph representation used in `PyG`. To add more properties to a graph, e.g. node or edge weights, we need to wrap the `edge_index` with a `Data` object:

In [ ]:
g = Data(edge_index=edge_index)

g["node_weight"] = tensor([1, 1, 1, 0, 0, 0, 0])
g["edge_weight"] = tensor([1, 2, 1, 1, 1, 2, 1, 1, 1])

### 2. Graph Visualisations in PathPyG

For visualisation, we need to create a `pathpyG.Graph`-object. We can visualise the node weights - that we can access using the `data`-variable - for example as different colors:

In [ ]:
net = Graph(g, mapping=IndexMap(["a", "b", "c", "d", "e", "f", "g"]))

colors = array(["red", "blue"])
plot(net, node_color=colors[net.data["node_weight"]].tolist(), node_label=list(net.nodes))

As you can see, the above graph is directed. To create an undirected graph, you can use:

In [ ]:
net = net.to_undirected()

colors = array(["red", "blue"])
plot(net, node_color=colors[net.data["node_weight"]].tolist(), node_label=list(net.nodes))

### 3. Random Walks

As you learned in the lecture, we need to sample random walks on the graph so that we can apply node2vec. We can use `PathPyG` for this as follows:

In [ ]:
def sample_random_walks(net, num_walks, walk_length):
    rw = RandomWalk(net)
    data = rw.run_experiment(runs=num_walks, steps=walk_length)
    walk_data = rw.get_paths(data)
    walks = [walk_data.get_walk(i) for i in range(walk_data.num_paths)]
    return walks

In [ ]:
walks = sample_random_walks(net, num_walks=2, walk_length=6)
walks

Note that for simplicity, we will only implement random walks with $p=1$ and $q=1$, thus, essentially only DeepWalk and not Node2Vec. The Jupyter Notebook 3 of Week 10 from the ML4Nets course that is also referenced as practice session in the lecture explores how to implement random walk sampling with different values for $p$ and $q$.

## Tasks

### 1. Continuous Bag-of-Words Model

The original paper that introduces Word2Vec ([T. Mikolov et al.](https://arxiv.org/pdf/1301.3781)) proposes multiple models to optimize the embedding. You learned about one option - the so-called Skip-Gram Model - in the lecture. In this task, we explore the alternative: The Continuous Bag-of-Words Model (CBOW)

#### a) What is CBOW?

Check out the referenced paper or other resources to learn about CBOW. Explain how it works and what the differences to Skip-Gram are with your own words. What does this mean in the graph context?

*Insert CBOW explanation and differences to Skip-Gram here...*

#### b) Implement CBOW

##### Utility Functions
For the implementation, we can reuse some parts of the implementation from the notebooks presented in the lecture. Other methods need to be implemented differently. Complete the methods below so that they will work for CBOW. You can copy the code from the notebooks if suitable.

In [ ]:
def build_vocabulary(tokens):
    # TODO
    return NotImplementedError()

def build_word_context_tuples(tokens, window_size):
    # TODO
    return NotImplementedError()

def get_ohes(words, vocab):
    # TODO
    return NotImplementedError()

In [ ]:
# Sanity Checks
print("Walks:")
print(walks, "\n")

print("Vocabulary:")
vocab = build_vocabulary(walks)
print(vocab, "\n")

print("Word + Context:")
word_context_tuples = build_word_context_tuples(walks, window_size=2)
print(word_context_tuples, "\n")

print("One-Hot Encodings:")
print(get_ohes(word_context_tuples[0][0], vocab))
print(get_ohes(word_context_tuples[0][1], vocab))

#### CBOW Model

Define a `Torch`-`Module` that implements the CBOW model below:

In [ ]:
class CBOW(Module):

    def __init__(self, embedding_dim, vocab_size):
        super(CBOW, self).__init__()
        
        # TODO

    def forward(self, x):
        # TODO
        return NotImplementedError()


In [ ]:
# Sanity Check
cbow = CBOW(3, len(vocab))
probs = cbow(get_ohes(word_context_tuples[0][1], vocab))
assert len(vocab) == probs.size(1)

#### Training Loop

Define a training loop to train your model:

In [ ]:
walks = sample_random_walks(net, num_walks=100, walk_length=10)
vocab = build_vocabulary(walks)
word_context_tuples = build_word_context_tuples(walks, window_size=2)
model = CBOW(embedding_dim=5, vocab_size=len(vocab))

# TODO: Initialize loss function and optimizer

epochs = 50
losses = []
for i in range(epochs):
    l = 0
    # TODO: Inner Training Loop

    losses.append(l/len(word_context_tuples))
plt.plot(range(epochs), losses)

In [ ]:
# Sanity Check
def get_embedding(node, model, vocab):
    return model.embeddings.weight.data[:,vocab.index(node)]

svd = TruncatedSVD()
low_dim = svd.fit_transform(array([get_embedding(w, model, vocab).detach().numpy() for w in vocab]))

fig, ax = plt.subplots()
ax.scatter(low_dim[:,0], low_dim[:,1])

for i, txt in enumerate(vocab):
    plt.arrow(0, 0, low_dim[i,0], low_dim[i,1], color='orange', width=0.02, alpha=0.2)
    ax.annotate(txt, (low_dim[i,0]+0.02, low_dim[i,1]+0.02), fontsize=14)

### 2. Link Prediction

As explained in the lecture, the node embeddings created above can be used for a variety of tasks like node classification or link prediction.

#### Hadamard-based Link Prediction using Node Embeddings

The lecture briefly mentioned that we can use, e.g. the Hadamard product $x_i \cdot x_j$, as similarity score to predict the existence of edges given embedding vectors $x_i$ and $x_j$ for nodes $v_i$ and $v_j$. Complete the following code snippet that predicts if an edge exists between two nodes given their label using the Hadamard product.

In [ ]:
def predict_edge(node_i, node_j, model, vocab):
    # TODO
    return NotImplementedError()


In [ ]:
# Sanity Check
print("High expected score: ", predict_edge("a", "c", model, vocab))
print("Low expected score: ", predict_edge("a", "f", model, vocab))

#### Real-World Example

Evaluate the performance of your link predictor on the Karate Club network that you know from the lecture using the area under the curve of the receiver-operator characteristic. Since the network is comparably small, you can either compute the score for all possible node pairs or use a sample of non-existing links (negative sampling).

*Note: The Karate Club network is a particularly difficult example network for link prediction. Thus, the results will most likely be not very good. If you want, you can also choose a different example network.*

In [ ]:
karate = read_netzschleuder_network("karate", "77")
karate.mapping = IndexMap([str(i) for i in karate.data.node_name])

plot(karate, node_label=list(karate.nodes))